# 제 2회 KRX 주식 투자 알고리즘 경진대회

## Phase 1. Data

### 1. Data Import from pykrx packages : 2021-07-29 ~ 2023-07-28

In [464]:
### package importing
from pykrx import stock
from datetime import datetime, date, timedelta
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry
from itertools import groupby
from operator import itemgetter
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import explained_variance_score
from lightgbm import LGBMRegressor
from bs4 import BeautifulSoup

import xgboost
import requests
import numpy as np
import pandas as pd
import sklearn
import time
import io
import zipfile
import xml.etree.ElementTree as et
import json
import time
import os
import chardet
import warnings
warnings.filterwarnings('ignore')

In [3]:
### KRX제공 데이터 : 주식 종목코드 추출
data = pd.read_excel("C:/Users/kyt28/OneDrive/바탕 화면/submit_data/KRX 공모전 데이터/sample_submission.xlsx")
code = data['종목코드']
code = [code[i].replace("A","") for i in range(len(code))]

In [20]:
### pykrx package로부터 종목 데이터 추출

## Base 날짜 세팅
date = stock.get_market_ohlcv("20210729", "20230728", code[0], adjusted = False).reset_index()['날짜']

## 종목 데이터 추출 by pykrx
stock_data = pd.DataFrame()
th = 0
error_code = []

for i in code :
    try :
        dat = pd.merge(date, stock.get_market_ohlcv("20210729", "20230728", i, adjusted = False).reset_index(), on = '날짜', how = 'left')
        time.sleep(0.5)
        dat = pd.merge(dat, stock.get_market_fundamental("20210729", "20230728", i).reset_index(), on = '날짜', how = 'left')
        time.sleep(0.5)
        dat = pd.merge(dat, stock.get_market_trading_value_by_date("20210729", "20230728", i, detail=True).reset_index(), on = '날짜', how = 'left')
        time.sleep(0.5)
        dat = pd.merge(dat, stock.get_market_cap("20210729", "20230728", i).reset_index().drop(['거래량','거래대금'],axis=1), on = '날짜', how = 'left')
        time.sleep(0.5)
        dat = pd.merge(dat, stock.get_exhaustion_rates_of_foreign_investment("20210729", "20230728", i).reset_index().drop(['상장주식수'],axis=1), on = '날짜', how = 'left')

        stock_data['종목코드'] = i
        
        stock_data = pd.concat([stock_data,dat], axis=0)
        
    except :
        error_code.append(i)

In [70]:
### error code

## 에러코드는 데이터 추출이 불가능하므로 분석대상에서 제외 (포트폴리오 대상에서 제외 (201~1800))
ec = ["A"+error_code[i] for i in range(len(error_code))]
print('에러코드 갯수 :',len(ec))
print('에러코드 :',ec)

에러코드 갯수 : 41
에러코드 : ['A003560', 'A006580', 'A007610', 'A012600', 'A014200', 'A015540', 'A024810', 'A032860', 'A033180', 'A033340', 'A044060', 'A053590', 'A056090', 'A057880', 'A062860', 'A065560', 'A066410', 'A069110', 'A078130', 'A078590', 'A091090', 'A093230', 'A096040', 'A099520', 'A101140', 'A109070', 'A121800', 'A136510', 'A150840', 'A160600', 'A178780', 'A181340', 'A214870', 'A215090', 'A226440', 'A257370', 'A263540', 'A268600', 'A269620', 'A290380', 'A318020']


In [98]:
## 에러코드에 대한 순위 마지막순번으로 부여 x ~ 1800위
error_rank = pd.DataFrame(ec,columns = ['종목코드'])

rank = []
for i in range(1801-len(error_rank),1801) :
    rank.append(i)
    
error_rank['순위'] = rank

In [99]:
## data export
error_rank.to_csv("C:/Users/kyt28/OneDrive/바탕 화면/submit_data/phase1_result.csv", index=False, encoding="utf-8-sig")

### 2. Data Export

In [81]:
### 최종 분석 대상 종목코드의 데이터

## 사용 변수 추출
stock_data = stock_data[['날짜', '종목코드', '종가', '저가', '고가', '거래량', '거래대금', '등락률', 'BPS', 'PER','PBR', 'EPS', '연기금', '외국인', '기타외국인', '시가총액', '상장주식수', '지분율']]

## data shape
print(stock_data.shape)

## 분석 대상 종목 갯수
print(stock_data['종목코드'].nunique())

## 종목코드 복구
stock_data['종목코드'] = "A" + stock_data["종목코드"]

(280137, 18)
1959


In [257]:
### Date Split & Export

## 모델에 활용할 데이터 (2021-07-29 ~ 2023-07-28) : 거래정지 일수(거래량 = 0)가 많은 종목들을 선별하기 위한 데이터 셋
notrade = stock_data[['날짜','종목코드','거래량']]
notrade.to_csv("C:/Users/kyt28/OneDrive/바탕 화면/submit_data/notrade.csv", index=False, encoding="utf-8-sig")

## 분석에 활용할 데이터 (2023-01-01 ~ 2023-07-28)
stock_data = stock_data[stock_data['날짜'] >= '2023-01-01']
stock_data.to_csv("C:/Users/kyt28/OneDrive/바탕 화면/submit_data/phase1_data.csv", index=False, encoding="utf-8-sig")

---------------------------------------------------------------------------------------------------------------------------------------------------------------------

## Phase 2. Data Variable
: 해당 Phase는 분석에 필요한 다양한 변수들을 만드는 작업을 거친다

### 1. 변수 생성

In [101]:
### data import
data = pd.read_csv("C:/Users/kyt28/OneDrive/바탕 화면/submit_data/phase1_data.csv")

## 변수 확인
data.columns

## unique한 종목코드 추출
code = data['종목코드'].unique().tolist()

##### 변수 생성에 필요한 함수

In [105]:
def MA(n,x) :
    ma = []
    
    for i in code :
        ma.extend(data[data['종목코드'] == i][x].rolling(window=n).mean().tolist())
    
    return ma

In [115]:
def MSD(n,x) :
    msd = []
    
    for i in code :
        msd.extend(data[data['종목코드'] == i][x].rolling(window=n).std().tolist())
    
    return msd

##### - Y 변수 : 개별 종목 당 현재 날짜 기준으로 15영업일 뒤의 Sharpe Index 계산

In [102]:
### Y
stock_SI = []

for c in code :
    dat = data[data['종목코드'] == c]
    dat_end = dat['종가'].tolist()
    dat_re = dat['등락률'] / 100 * 250 
    dat_re = dat_re.tolist()
    for i in range(14,(len(dat)-1)) :
        rev = (dat_end[i+1] - dat_end[i+1-14])/dat_end[i+1-14] * 250/15 - 0.035
        d = dat_re[(i-12):(i+1)]
        vol = np.sqrt(np.sum((d - np.mean(d))**2)/13)
        stock_SI.append(rev/vol)
    stock_SI.extend([np.nan]*15)
    
data['15by1_SI'] = stock_SI
data = data.replace([np.inf,-np.inf],0)

<ipython-input-102-c5a08351e33a>:13: RuntimeWarning: divide by zero encountered in double_scalars
  stock_SI.append(rev/vol)


##### X 변수

##### - X1 : 외인 & 연기금 수급 by 거래대금

In [106]:
data['외국인총합'] = data['외국인'] + data['기타외국인']
data['외국인거래율'] = data['외국인총합']/data['거래대금']
data['연기금거래율'] = data['연기금']/data['거래대금'] 

##### - X2 : 외인 거래 대비 5일 이동평균선(외인 거래)

In [107]:
data['MAfor'] = MA(5,'외국인총합')
data['ForProp'] = data['외국인총합']/data['MAfor']

##### - X3 : 이동평균선 (5일, 15일, 60일)

In [109]:
data['MA5'] = MA(5,'종가')
data['MA15'] = MA(15,'종가')
data['MA60'] = MA(60,'종가')

##### - X4 : MACD (12일, 26일 차이 & signal 선)

In [112]:
data['MA12'] = MA(12,'종가')
data['MA26'] = MA(26,'종가')
data['MACD'] = data['MA12'] - data['MA26']
data['MACDsignal'] = MA(9,'MACD')

In [113]:
# MACD가 MACD signal 선을 상향 돌파하면 high, 하향 돌파하면 low
macd = []
for c in code :
    dt = data[data['종목코드'] == c].reset_index()
    
    macd.append(np.nan)
    for i in range(len(dt)-1) :
        if pd.isna(dt['MACDsignal'][i+1]) :
            macd.append(np.nan)
        elif (dt['MACD'][i] < dt['MACDsignal'][i]) & (dt['MACD'][i+1] > dt['MACDsignal'][i+1])  :
            macd.append("high")
        elif (dt['MACD'][i] > dt['MACDsignal'][i]) & (dt['MACD'][i+1] < dt['MACDsignal'][i+1]) :
            macd.append("low")
        else : macd.append("Normal")
            
data['MACDtouch'] = macd
data = data.drop(['MA12','MA26'],axis=1)

##### - X5 : 볼린저밴드 (15,2)

In [116]:
### 볼린저 밴드 : 종가가 15이평선의 2표준편차 이상 터치시 HighTouch, 하향 터치시 LowTouch
MA15sd = MSD(15,'종가')

bollinger = []
for i in range(len(data)) :
    if pd.isna(data['MA15'][i]) :
        bollinger.append(np.nan)
    elif data['MA15'][i] - MA15sd[i]*2 > data['종가'][i] :
        bollinger.append("LowTouch")
    elif data['MA15'][i] + MA15sd[i]*2 < data['종가'][i] :
        bollinger.append("HighTouch")
    else : bollinger.append("Normal")
        
data['bollinger'] = bollinger

##### - X6 : RSI

In [117]:
### RSI 지수

## 전날 대비 상승 or 하락분 계신
data['dff'] = data['종가']

dff = []
for i in code :
    dt = data[data['종목코드'] == i]['dff']
    dff.extend(dt.diff(periods=1).tolist())
    
data['dff'] = dff

In [118]:
## RSI 계산
RSI = []
for i in code :
    dt = data[data['종목코드'] == i]['dff'].tolist()
    RSI.extend([np.nan]*14)
    for j in range(len(dt)-14) :
        inc = dec = 0
        for k in range(j,j+14) :
            if dt[k] > 0 : inc += dt[k]
            elif dt[k] < 0 : dec += abs(dt[k])
        if inc+dec == 0 :
            RSI.append(0.5)
        else : RSI.append(inc/(inc + dec) * 100)
            
data['RSI'] = RSI
data = data.drop(['dff'],axis=1)

##### - X7 : MFI

In [119]:
### MFI 지수

## 종가 저가 고가 평균 및 거래량 곱 계산
data['MF'] = (data['고가'] + data['저가'] + data['종가'])/3 * data['거래량']
data['TP'] = (data['고가'] + data['저가'] + data['종가'])/3

## 평균값의 상승분 계산
dff_tp = []
for i in code :
    dt = data[data['종목코드'] == i]['TP']
    dff_tp.extend(dt.diff(periods=1).tolist())
    
data['TP_dff'] = dff_tp

In [122]:
## MFI 계산
MFI = []
for i in code :
    dff_tp = data[data['종목코드'] == i]['TP_dff'].tolist()
    mf = data[data['종목코드'] == i]['MF'].tolist()
    MFI.extend([np.nan]*14)
    for j in range(len(dff_tp)-14) :
        inc = dec = 0
        for k in range(j,j+14) :
            if dff_tp[k] > 0 : inc += mf[k]
            elif dff_tp[k] < 0 : dec += mf[k]
        if dec == 0 :
            MFI.append(50)        
        else : MFI.append(100 - 100/(1 + inc/dec))
            
data['MFI'] = MFI
data = data.drop(['MF','TP_dff'],axis=1)

##### X8 : CCI

In [123]:
### CCI 지수

## TP이평선 및 차이 절댓값 계산
data['m'] = MA(15,'TP')
data['d'] = abs(data['TP'] - data['m'])
data['d'] = MA(15,'d')

## CCI 계산
data['CCI'] = (data['TP'] - data['m'])/(0.015 * data['d'])
data = data.drop(['m','d','TP'], axis = 1)

### 2. Data Export

In [129]:
data.to_csv("C:/Users/kyt28/OneDrive/바탕 화면/submit_data/phase2_data.csv", index=False, encoding="utf-8-sig")

---------------------------------------------------------------------------------------------------------------------------------------------------------------------

## Phase 3. Stock Except Decision
: Phase 3는 분석 대상 중에서 먼저, 다양한 지표들을 활용하여 강력한 Long signal 또는 Short signal에 따라 미리 포트폴리오에 순위를 지정하는 작업이다
- Long : 최근 5영업일 ~ 10영업일에 대해 Long포지션에 맞는 다양한 지수로 포트폴리오에 미리 선정하는 작업
- Short : 최근 5영업일 ~ 10영업일에 대해 Short포지션에 맞는 다양한 지수로 포트폴리오에 미리 선정하는 작업

### 1. Data Import

In [155]:
### data import
data = pd.read_csv("C:/Users/kyt28/OneDrive/바탕 화면/submit_data/phase2_data.csv", low_memory = False)

## 불필요한 변수 제거
data = data.drop(['저가','고가','외국인','기타외국인','시가총액','연기금','상장주식수'],axis=1)

### 2. Exception Decision

##### 2-0. 거래정지종목 : private 기간 중 마지막 영업일 (2023년 7월 28일)에 정지된 종목

In [156]:
### 거래정지 종목

## 거래정지 종목 추출
excep0_code = data[(data['날짜'] == '2023-07-28') & (data['거래량'] == 0)]['종목코드'].tolist()
print('거래정지 종목코드 개수 :',len(excep0_code))
print('거래정지 종목코드 :',excep0_code)

## 거래정지 종목 제외
data = data[~data['종목코드'].isin(excep0_code)]

## unique 종목코드 추출
code = data['종목코드'].unique().tolist()

거래정지 종목코드 개수 : 5
거래정지 종목코드 : ['A001340', 'A047820', 'A048260', 'A052670', 'A102280']


##### 2-1. RSI & MACD 
- Long = 5영업일 내에 RSI지수가 30이하에서 30이상으로 돌파하고 MACD선이 signal선 상향 돌파 후 유지
- Short = 5영업일 내에 RSI지수가 70이상에서 70이하로 돌파하고 MACD선이 signal선 하향 돌파 후 유지

In [157]:
### RSI & MACD 

## base setting
new_data = data[data['날짜'] >= '2023-07-21']
len(new_data)/1954

6.0

In [158]:
## Long
excep1_long = []

for c in code :
    dt = new_data[new_data['종목코드'] == c].reset_index(drop=True)
    
    cnt = 0
    for i in range(1,5) :
        if (dt['MACD'][i] < dt['MACDsignal'][i]) & (sum(dt['MACD'][(i+1):] > dt['MACDsignal'][(i+1):]) == 5-i) : cnt += 1
            
    if (np.sum(dt['RSI'][0] <= 30) == 1) & (np.sum(dt['RSI'][1:] > 30) == 5) & (cnt >= 1) :
        excep1_long.append(c)
        
print(excep1_long,'\n',len(excep1_long))

['A003060', 'A005180', 'A011330', 'A040420', 'A078140', 'A156100', 'A214450', 'A241820', 'A243070', 'A246710'] 
 10


In [159]:
## Short
excep1_short = []

for c in code :
    dt = new_data[new_data['종목코드'] == c].reset_index(drop=True)
    
    cnt = 0
    for i in range(1,5) :
        if (dt['MACD'][i] > dt['MACDsignal'][i]) & (sum(dt['MACD'][(i+1):] < dt['MACDsignal'][(i+1):]) == 5-i) : cnt += 1
            
    if (np.sum(dt['RSI'][0] >= 70) == 1) & (np.sum(dt['RSI'][1:] < 70) == 5) & (cnt >= 1) :
        excep1_short.append(c)
        
print(excep1_short,'\n',len(excep1_short))

['A001470', 'A002200', 'A059090', 'A158430', 'A200710', 'A211050', 'A262260', 'A321260'] 
 8


##### 2-2. MFI 
- Long = 5일 간 80 이하였다가 5일 동안 80 이상 유지
- Short = 5일간 20 이상이었다가 5일 동안 20 이하 유지

In [162]:
### MFI

## base setting
new_data = data[data['날짜'] >= '2023-07-17']
len(new_data)/1954

10.0

In [163]:
## Long
excep2_long = []

for c in code :
    dt = new_data[new_data['종목코드'] == c].reset_index(drop=True)

    if (sum(dt['MFI'][:5] <= 80) + sum(dt['MFI'][5:] > 80)) == 10 :
        excep2_long.append(c)
        
print(excep2_long,'\n',len(excep2_long))

['A007460', 'A109960', 'A131030'] 
 3


In [166]:
## short
excep2_short = []

for c in code :
    dt = new_data[new_data['종목코드'] == c].reset_index(drop=True)

    if (sum(dt['MFI'][:5] >= 20) + sum(dt['MFI'][5:] < 20)) == 10 :
        excep2_short.append(c)
        
print(excep2_short,'\n',len(excep2_short))

['A001520', 'A002450', 'A004060', 'A005670', 'A005680', 'A009160', 'A010280', 'A012610', 'A013720', 'A015860', 'A033130', 'A037460', 'A037760', 'A042370', 'A045520', 'A045660', 'A054800', 'A056700', 'A066130', 'A067990', 'A079430', 'A080580', 'A085810', 'A089150', 'A099440', 'A100660', 'A119850', 'A148250', 'A153460', 'A187270', 'A263020', 'A290120'] 
 32


##### 2-3. 볼린저밴드
- Long : 5영업일 내에 Normal에서 HighTouch로 변경되고 유지한 경우
- Short : 5영업일 내에 Normal에서 LowTouch로 변경되고 유지한 경우

In [167]:
### 볼린저밴드

## base setting
new_data = data[data['날짜'] >= '2023-07-21']
len(new_data)/1954

6.0

In [168]:
## Long
excep3_long = []

for c in code :
    dt = new_data[new_data['종목코드'] == c].reset_index(drop=True)
    
    for i in range(1,4) :
        if (sum(dt['bollinger'][i:] == "HighTouch") == 6-i) & (sum(dt['bollinger'][:i] == "Normal") == i) :
            excep3_long.append(c)
        
print(excep3_long,'\n',len(excep3_long))

['A078140'] 
 1


In [169]:
## Short
excep3_short = []

for c in code :
    dt = new_data[new_data['종목코드'] == c].reset_index(drop=True)
    
    for i in range(1,4) :
        if (sum(dt['bollinger'][i:] == "LowTouch") == 6-i) & (sum(dt['bollinger'][:i] == "Normal") == i) :
            excep3_short.append(c)
        
print(excep3_short,'\n',len(excep3_short))

['A003800', 'A078000', 'A107590', 'A290670'] 
 4


##### 2-4. CCI 
- Long = 5영업일 전부터 3일 이상 100이하였다가 100을 돌파 후 유지인 경우
- Short = 5영업일 전부터 3일 이상 -100이상이었다가 -100 이하 돌파 후 유지인 경우

In [170]:
### CCI

## base setting
new_data = data[data['날짜'] >= '2023-07-21']
len(new_data)/1954

6.0

In [171]:
## Long
excep4_long = []

for c in code :
    dt = new_data[new_data['종목코드'] == c].reset_index(drop=True)
    
    if (sum(dt['CCI'][:3] <= 100) == 3) & (sum(dt['CCI'][3:] > 100) == 3) :
        excep4_long.append(c)
            
print(excep4_long,'\n',len(excep4_long))

['A009730', 'A011330', 'A031980', 'A078140', 'A105560', 'A134380'] 
 6


In [172]:
## Short
excep4_short = []

for c in code :
    dt = new_data[new_data['종목코드'] == c].reset_index(drop=True)

    if (sum(dt['CCI'][:3] >= -100) == 3) & (sum(dt['CCI'][3:] < -100) == 3) :
        excep4_short.append(c)

print(excep4_short,'\n',len(excep4_short))

['A000390', 'A000440', 'A001550', 'A002140', 'A003120', 'A003310', 'A004250', 'A006390', 'A007690', 'A007860', 'A008260', 'A009470', 'A010170', 'A010960', 'A011700', 'A012320', 'A013700', 'A014830', 'A018620', 'A019210', 'A019660', 'A023450', 'A025860', 'A025880', 'A027360', 'A030190', 'A034310', 'A037710', 'A038110', 'A041960', 'A043910', 'A048530', 'A049770', 'A051630', 'A053280', 'A053690', 'A053950', 'A060150', 'A060280', 'A064090', 'A064520', 'A065710', 'A067630', 'A078890', 'A083420', 'A091340', 'A092200', 'A092230', 'A093640', 'A107590', 'A108380', 'A108490', 'A120110', 'A131400', 'A140520', 'A142210', 'A170790', 'A195990', 'A196300', 'A203450', 'A219550', 'A221980', 'A222080', 'A241520', 'A256940', 'A290670', 'A297090', 'A298000', 'A317400', 'A317690', 'A322000', 'A330350', 'A336260', 'A348030', 'A348150'] 
 75


##### 2-5. MA 
- Long = 3영업일 동안 15이평선이 60이평선 아래 & 이후 3영업일 동안 15이평선이 60이평선 위
- Short = 3영업일 동안 15이평선이 60이평선 위 & 이후 3영업일 동안 15이평선이 60이평선 아래

In [173]:
### MA

## base setting
new_data = data[data['날짜'] >= '2023-07-21']
len(new_data)/1954

6.0

In [174]:
## Long
excep5_long = []

for c in code :
    dt = new_data[new_data['종목코드'] == c].reset_index(drop=True)

    if (sum(dt['MA15'][:3] <= dt['MA60'][:3]) == 3) & (sum(dt['MA15'][3:] > dt['MA60'][3:]) == 3) :
        excep5_long.append(c)

print(excep5_long,'\n',len(excep5_long))

['A267250', 'A300080'] 
 2


In [175]:
## Short
excep5_short = []

for c in code :
    dt = new_data[new_data['종목코드'] == c].reset_index(drop=True)

    if (sum(dt['MA15'][:3] >= dt['MA60'][:3]) == 3) & (sum(dt['MA15'][3:] < dt['MA60'][3:]) == 3) :
        excep5_short.append(c)

print(excep5_short,'\n',len(excep5_short))

['A002150', 'A002900', 'A004710', 'A006880', 'A009680', 'A010580', 'A011790', 'A019770', 'A021820', 'A023800', 'A024910', 'A025540', 'A032560', 'A036030', 'A036170', 'A037330', 'A041830', 'A043260', 'A047810', 'A051370', 'A051380', 'A052460', 'A054050', 'A057680', 'A058730', 'A059120', 'A060980', 'A071090', 'A072470', 'A086390', 'A091700', 'A126640', 'A126880', 'A128820', 'A129260', 'A142210', 'A153490', 'A171120', 'A180640', 'A192400', 'A214420', 'A214680', 'A220180', 'A245620', 'A298050', 'A307280', 'A309930', 'A310200', 'A311390', 'A352480'] 
 50


##### 2-6. 외국인 
- Long = 10영업일 동안 꾸준하게 외인 매수
- Short = 10영업일 동안 꾸준하게 외인 매도

In [176]:
### 외국인

## base setting
new_data = data[data['날짜'] >= '2023-07-17']
len(new_data)/1954

10.0

In [177]:
## Long
excep6_long = []

for c in code :
    dt = new_data[new_data['종목코드'] == c].reset_index(drop=True)
        
    if sum(dt['외국인총합'] > 0) == 10 :
        excep6_long.append(c)

print(excep6_long,'\n',len(excep6_long))

['A002030', 'A003410', 'A004170', 'A005940', 'A009780', 'A032580', 'A034950', 'A037710', 'A081660', 'A084370', 'A263810', 'A294140'] 
 12


In [178]:
## Short
excep6_short = []

for c in code :
    dt = new_data[new_data['종목코드'] == c].reset_index(drop=True)
        
    if sum(dt['외국인총합'] < 0) == 10 :
        excep6_short.append(c)

print(excep6_short,'\n',len(excep6_short))

['A001290', 'A004780', 'A005490', 'A005710', 'A005990', 'A008370', 'A018120', 'A020000', 'A021650', 'A021820', 'A024830', 'A025000', 'A029780', 'A030000', 'A031510', 'A034220', 'A034310', 'A034590', 'A036670', 'A038390', 'A039570', 'A044820', 'A045060', 'A049120', 'A064850', 'A066620', 'A067830', 'A078070', 'A217730', 'A260970'] 
 30


### 3. Data Export

In [237]:
### Long, Short 포트폴리오에 지정될 변수들을 제외하고 모델에 사용할 데이터 export

## 제외변수 통합
except_code = excep1_long + excep2_long + excep3_long + excep4_long + excep5_long + excep6_long + excep1_short + excep2_short + excep3_short + excep4_short + excep5_short + excep6_short + excep0_code
except_code = list(set(except_code))
print('제외된 변수 갯수 :',len(except_code))

## data export
data = data[~data.종목코드.isin(except_code)].reset_index(drop=True)
data.to_csv("C:/Users/kyt28/OneDrive/바탕 화면/submit_data/phase3_data.csv", index=False, encoding="utf-8-sig")

제외된 변수 갯수 : 229


### 4. Long, Short Portfolio
- 앞서, 선정된 Long과 Short에 속한 주식들을 미리 포트폴리오에 선정
- 숫자는 포트폴리오 선정의 중요도를 뜻함 (ex. 외국인은 MA보다 중요함)

In [190]:
### Position

## Long
long = set(excep1_long + excep2_long + excep3_long + excep4_long + excep5_long + excep6_long)
print('Long Position 갯수 :',len(long),'\n')

## Short
short = set(excep1_short + excep2_short + excep3_short + excep4_short + excep5_short + excep6_short)
print('Short Position 갯수 :',len(short),'\n')

## Intersection : 어느 파트에서 겹쳐졌는지 확인
same_code = list(short.intersection(long))
print('중복 Position 종목 :',same_code,'\n')

long_same = []
short_same = []
for c in same_code :
    for i in range(1,7) :
        if c in globals()['excep{}_long'.format(i)] :
            long_same.append('excep'+str(i)+'_long')
        if c in globals()['excep{}_short'.format(i)] :
            short_same.append('excep'+str(i)+'_short')    

print('중복 종목코드 Position :',long_same)
print('중복 종목코드 Position :',short_same)

Long Position 갯수 : 31 

Short Position 갯수 : 194 

중복 Position 종목 : ['A037710'] 

중복 종목코드 Position : ['excep6_long']
중복 종목코드 Position : ['excep4_short']


In [191]:
## 중복된 경우, 중요도에 의해서 선정
short.remove('A037710')

##### 4-1. Long, Short Portfolio Rank

In [229]:
### Rank

## Long Position : 1 ~
l = pd.DataFrame(set(long))
l.columns = ['종목코드']

rank = []
for i in range(len(l)) :
    rank.append(i+1)
    
l['순위'] = rank

## Short Position : ~ 2000
s = pd.DataFrame(set(short))
s.columns = ['종목코드']

rank = []
for i in range(len(s),0,-1) :
    rank.append(2000-i+1)
    
s['순위'] = rank

## concat
portfolio = pd.concat([l,s],axis = 0)
portfolio = portfolio.reset_index(drop=True)
portfolio

,종목코드,순위
0,A084370,1
1,A031980,2
2,A037710,3
3,A009730,4
4,A003060,5
...,...,...
219,A024830,1996
220,A039570,1997
221,A006880,1998
222,A219550,1999


##### 4-2. Exception code Portfolio Rank

In [230]:
## Exception code Position

# Phase 3에서 제외한 excep_code
excep_data1 = pd.DataFrame(set(excep0_code))
excep_data1.columns = ['종목코드']

# Phase 1에서 제외했던 error_rank 추출 후, concat
excep_data2 = pd.read_csv("C:/Users/kyt28/OneDrive/바탕 화면/submit_data/phase1_result.csv")
first_rank = excep_data2['순위'][0]

# data merge
excep = pd.concat([excep_data1,excep_data2],axis=0).reset_index(drop=True)
excep.loc[excep['순위'].isna(),'순위'] = [first_rank - i for i in range(len(excep0_code),0,-1)]
excep

,종목코드,순위
0,A047820,1755.0
1,A102280,1756.0
2,A001340,1757.0
3,A048260,1758.0
4,A052670,1759.0
5,A003560,1760.0
6,A006580,1761.0
7,A007610,1762.0
8,A012600,1763.0
9,A014200,1764.0


##### 4-3. Portfolio Export

In [234]:
## portfolio Merge
phase3_portfolio = pd.concat([portfolio,excep],axis=0).sort_values(by='순위')
phase3_portfolio.to_csv("C:/Users/kyt28/OneDrive/바탕 화면/submit_data/phase3_result.csv", index=False, encoding="utf-8-sig")
phase3_portfolio

,종목코드,순위
0,A084370,1.0
1,A031980,2.0
2,A037710,3.0
3,A009730,4.0
4,A003060,5.0
...,...,...
219,A024830,1996.0
220,A039570,1997.0
221,A006880,1998.0
222,A219550,1999.0


---------------------------------------------------------------------------------------------------------------------------------------------------------------------

## Phase 4. Model Decision
: Phase 4는 미리 선정된 주식들을 제외하고 나머지 주식들에 대해서 다양한 머신러닝 모형을 통해 15일 뒤의 샤프지수를 예측하는 모형을 구축하여 나머지 순위를 할당하는 작업을 한다

### 1. Data Import

In [552]:
### Data Import
data = pd.read_csv("C:/Users/kyt28/OneDrive/바탕 화면/submit_data/phase3_data.csv", low_memory = False)

### 2. 공시 데이터

##### 2-1. Dart & KIND Data 추출에 필요한 함수 : 코드키 및 함수 define

In [553]:
### 코드키 및 함수 define

## 코드 키
crtfc_key = # 개인 API KEY

## 함수

# 데이터 내 날짜별로 정수에 대응시키는 함수 : ex) 2021-06-01 ~ 2023-05-31 -> 0 ~ 494
def date_to_index(row) :
    return(dates_index[row["날짜"]])

#  kind로 부터 주가의 등락의 긍/부정적 영향을 미치는 공시를 찾아 긍/부정 변수 부여
def count_positive(row):
    if "신주인수권" not in row["공시제목"]:
        return sum(row['공시제목'].count(term) for term in positive)
    else:
        return 0
def count_negative(row):
    return sum(row['공시제목'].count(term) for term in negative)


# 거래정지 첫 날만 남기기(연속되는 날짜는 같은 거래정지 사유라는 가정 하에 진행)

def start_halt(row) :
    dates_list = zero_dates[row["종목코드"]]
    if row["날짜1"] == dates_list[-1][0] :
        return True
    else :
        return False

    
# 거래정지사유가 주식병합인지 분할인지 Dart 공시를 통해 확인 -> 병합이나 분할의 경우, 거래정지 시작일과 종료일 확인

def check_halt_range(corp_code, date0, crtfc_key) : # 특정 기업의 특정 날짜 기준 최근 1년 간 주식병합/분할 관련 공시확인
    
    # text 내 여러 단어들 각각 다른 단어로 교체
    def replace_multiple(text, dic):
        for i, j in dic.items():
            text = text.replace(i, j)
        return text
    
    s = requests.Session()
    retries = Retry(total=5, backoff_factor=0.1, status_forcelist=[500, 502, 503, 504])
    s.mount('https://', HTTPAdapter(max_retries=retries))
    
    date1 = date.fromisoformat(date0)
    
    s_date = str(date1 - timedelta(days = 365)).replace("-", "")
    e_date = date0.replace("-", "")
    
    time.sleep(1)
    params = {"crtfc_key" : crtfc_key,
              "corp_code" : corp_code,
              "bgn_de" : s_date,
              "end_de" : e_date,
              "pblntf_ty" : "I",
              "page_no" : 1,
              "page_count" : 30}
    
    url = f"https://opendart.fss.or.kr/api/list.json"
    response = s.get(url, params = params, verify = False)
    
    if json.loads(response.text)["status"] == "000" :
        lists = json.loads(response.text)['list']
        for gongsi in lists :
            if ("주식분할" in gongsi["report_nm"] or "주식병합" in gongsi["report_nm"]) and "철회" not in gongsi["report_nm"] :
                    
                # 공시가 주식병합/분할 관련 공시일 경우, 해당 공시문서를 통해 거래정지시작일과 종료일 반환
                
                params = {"crtfc_key" : crtfc_key, "rcept_no" : gongsi["rcept_no"]}

                rcept_no = params["rcept_no"]
                doc_zip_path = os.path.abspath(f'./document_{rcept_no}.zip')

                url = "https://opendart.fss.or.kr/api/document.xml"
                response = s.get(url, params = params, verify = False)
                with open(doc_zip_path, 'wb') as fp:
                    fp.write(response.content)

                with zipfile.ZipFile(doc_zip_path, 'r') as zf:
                    zf.extractall()
                    filename = zf.infolist()[0].filename
                
                rawdata = open(filename, 'rb').read()
                result = chardet.detect(rawdata)
                encoding = result['encoding']
                    
                with open(filename, 'r', encoding = encoding) as fp:
                    lines = fp.readlines()

                replacements = {'&cr;': '\n', '<주': '<주', 'M&A': 'M&A', 'R&D': 'R&D'}
                lines = [replace_multiple(line, replacements) for line in lines]

                with open('./temp.xml', 'w', encoding = 'utf-8') as fp:
                    fp.writelines(lines)

                soup = BeautifulSoup(''.join(lines), 'lxml')

                trade_halt_row1 = soup.find_all(lambda tag: tag.name == 'td' and tag.text.__contains__("매매거래정지"))[-1]
                start_date = trade_halt_row1.find_next_siblings('td')[-1].text.split()[-1]

                trade_halt_row2 = soup.find_all(lambda tag: tag.name == 'tr' and tag.text.__contains__("매매거래정지"))[-1]
                end_date = trade_halt_row2.find_next_siblings('tr', limit=1)[0].text.split()[-1]

                if "주식분할" in gongsi["report_nm"] :
                    return ("주식분할", start_date, end_date)
                elif "주식병합" in gongsi["report_nm"] :
                    return ("주식병합", start_date, end_date)
            
    return(1, "-", "-")

##### 2-2. Dart 데이터 추출
- 각 종목별로 2023년 7월 28일 기준으로 주식 병합 또는 분할을 추출
- 실제 Private 기간 안에 분할, 병합으로 인해 거래정지 되었던 주식이 풀릴 경우, 해당 종목코드는 포트폴리오 제외
- [4. Model Base Setting] 파트에서, 모든 분석 대상이 정해지고 난 후 7월 28일 기준으로 추출하여 해당되는 종목은 제외

##### 2-3. KIND 데이터 추출
- 분석기간인 2023년 5월 1일부터 2023년 7월 28일까지 KIND 공시 제목을 통한 긍정/부정 사진제작

In [554]:
### Data Import
kind = pd.read_csv("C:/Users/kyt28/OneDrive/바탕 화면/submit_data/kind data/상세검색.csv")

for i in range(1,25) :
    xls = pd.read_csv(f"C:/Users/kyt28/OneDrive/바탕 화면/submit_data/kind data/상세검색 ({i}).csv")
    kind = pd.concat([kind, xls], axis = 0)

In [555]:
### Data 조정
kind['시간'] = kind['시간'].str.split(expand = True)[0]
kind = kind.loc[:,["시간","종목코드", "공시제목"]]
kind = kind.dropna()
kind['종목코드'] = "A" + kind['종목코드'].apply(np.int64).apply(str).str.zfill(6)
kind.columns = ['날짜','종목코드','공시제목']
kind = kind.reset_index(drop=True)

In [556]:
### KIND 최종 긍정 부정

## 긍정/부정 사전
positive = ["감자결정", "감자완료", "주식소각", "신규상장"]
negative = ["사채원리금", "대출금", "횡령", "자본잠식", "불성실공시", "기타안내", "단기과열", "신주인수권"]

## Data Merge
kind['긍정'] = kind.apply(count_positive, axis = 1)
kind['부정'] = kind.apply(count_negative, axis = 1)

kind = kind[kind.긍정 + kind.부정 != 0].reset_index(drop = True)

kind = kind.groupby(["날짜", "종목코드"]).aggregate(sum).reset_index()

### 3. Data Preprocessing

##### - 거래정지가 21년 7월 29일 ~ 23년 7월 28일 기준으로 250영업일 이상 지속된 경우 제외 : 과도한 거래정지 일수는 기업 자체의 문제가 있다고 자체적으로 판단하여 포트폴리오에서 제외

In [557]:
### Phase1에서 2021년 7월 29일 ~ 2023년 7월 28일 데이터 Import
notrade = pd.read_csv("C:/Users/kyt28/OneDrive/바탕 화면/submit_data/notrade.csv")

## 거래량 0이 250일 이상 있는 종목 제거
no_trade = notrade[notrade['거래량'] == 0]
no_trade = no_trade['종목코드'].value_counts().reset_index()
excep_code1 = no_trade[no_trade['종목코드'] >= 250]['index'].tolist()
print('거래량 0이상이 250일 이상인 종목 갯수 :',len(excep_code1))

## Data에서 제거
data = data[~data['종목코드'].isin(excep_code1)]

거래량 0이상이 250일 이상인 종목 갯수 : 16


##### - 분석기간 : 3개월 기간 : 2023/5/1 ~ 2023/7/28

In [558]:
### Data Filtering & KIND긍정부정 데이터 합병
data = data[data['날짜'] >= '2023-05-01'].reset_index(drop = True)
data = pd.merge(data,kind,on = ['날짜','종목코드'], how = 'left')

## 긍정/부정 없는 데이터는 0으로 채우기
data[['긍정','부정']] = data[['긍정','부정']].fillna(0)

### 4. Model Base Setting

##### 4-1. Label Encoding

In [559]:
categ = data.dtypes[data.dtypes == "object"].index.tolist()
categ.remove('날짜')
categ.remove('종목코드')

label_encoder = LabelEncoder()
for feature in categ :
    data[feature] = label_encoder.fit_transform(data[feature])
    
data['종목코드_x'] = label_encoder.fit_transform(data['종목코드'])

##### 4-2. 7/28일 예측시점 제외 (마지막 예측할 기간)

In [560]:
real_test = data[data['날짜'] == '2023-07-28'].reset_index(drop=True)
real_test = real_test.drop(['날짜','15by1_SI'],axis=1)
data = data.drop('날짜',axis=1)

##### 4-3. 결측값 제거

In [561]:
### 결측값
print(data.isna().sum())
print('기존 데이터 row :',len(data))

종목코드              0
종가                0
거래량               0
거래대금              0
등락률               0
BPS              30
PER              30
PBR              30
EPS              30
지분율               0
15by1_SI      25785
외국인총합           307
외국인거래율          307
연기금거래율          307
MAfor           420
ForProp         471
MA5               0
MA15              0
MA60              0
MACD              0
MACDsignal        0
MACDtouch         0
bollinger         0
RSI               0
MFI               0
CCI               0
긍정                0
부정                0
종목코드_x            0
dtype: int64
기존 데이터 row : 104859


In [562]:
### 결측값 제거
data = data.dropna().reset_index(drop=True)
print('결측값 제거 데이터 row :',len(data))
print('종목코드 수 :',data['종목코드'].nunique())

결측값 제거 데이터 row : 78629
종목코드 수 : 1719


##### 4-4. 2023년 7월 28일 기준 Dart 주식분할/병합 공시 데이터

In [85]:
### 분석대상 코드 지정
code = data['종목코드'].unique().tolist()

# 각 기업별 데이터 마지막 날짜 기준(Private 기간 전 마지막 영업일) 최근 1년 간 주식병합/분할 관련 공시여부 확인 후 거래정지시작일과 종료일 반환
for i in range(len(code)) :
    if i % 20 == 0 :
        print(i)
    data.loc[data.종목코드 == code[i], ["거래정지사유", "거래정지시작일", "거래정지종료일"]] = check_halt_range(code[i], "2023-07-28", crtfc_key)

0
20
40
60
80
100
120
140
160
180
200
220
240
260
280
300
320
340
360
380
400
420
440
460
480
500
520
540
560
580
600
620
640
660
680
700
720
740
760
780
800
820
840
860
880
900
920
940
960
980
1000
1020
1040
1060
1080
1100
1120
1140
1160
1180
1200
1220
1240
1260
1280
1300
1320
1340
1360
1380
1400
1420
1440
1460
1480
1500
1520
1540
1560
1580
1600
1620
1640
1660
1680
1700


In [7]:
### 결과 확인 : Private 기간 안에 주식분할 또는 병합으로 인한 거래정지가 풀린 기업이 없기 떄문에 분석대상에 모두 포함
data = data[data['날짜'] == '2023-07-28']
data = data.drop('거래정지종료일',axis=1)
dat['거래정지종료일'].value_counts()

-             1700
2023-05-03       2
2023-08-24       1
2023-05-02       1
2023-04-12       1
2023-05-15       1
2023-05-11       1
2022-10-30       1
2023-04-21       1
2023-08-25       1
2023-05-24       1
2023-05-31       1
2023-05-22       1
2022-11-15       1
2023-09-11       1
2022-12-01       1
2023-05-10       1
2023-04-13       1
2022-12-09       1
Name: 거래정지종료일, dtype: int64

In [359]:
data = data.drop(["거래정지사유", "거래정지시작일", "거래정지종료일"],axis=1)

##### 4-5. 거래 분석기간 23년 5월 1일 ~ 23년 7월 28일 기준으로 40영업일 미만 데이터가 존재할 경우 분석 대상 제외

In [564]:
### 표본의 대표성에 의해 한 종목 당 최소 40개 이상의 데이터를 학습시킨다

## 40개 미만 데이터를 가지는 종목코드 추출
vc = data['종목코드'].value_counts().reset_index()
excep_code2 = vc[vc['종목코드'] < 40]['index'].tolist()

## 해당 종목코드 제외
data = data[~data['종목코드'].isin(excep_code2)].reset_index(drop=True)
real_test = real_test[~real_test['종목코드'].isin(excep_code2)].reset_index(drop=True)

##### 4-6. 모든 제외사항 후, Label Encoder 된 종목코드에 대한 dictionary

In [565]:
### dictionary
code_dict = data[['종목코드','종목코드_x']]
code_dict = code_dict.drop_duplicates('종목코드',keep = "first").reset_index(drop=True)

## Label Encoder에 맞게 종목코드 지정
data['종목코드'] = data['종목코드_x']
data = data.drop('종목코드_x',axis=1)

##### 4-7. data split

In [318]:
### Data Split
code = data['종목코드'].unique().tolist()
train = pd.DataFrame()
test = pd.DataFrame()

for i in code :
    dt = data[data['종목코드'] == i].reset_index(drop = True)
    tr, te = train_test_split(dt, train_size = int(len(dt) - 5), shuffle = False)
    
    train = pd.concat([train,tr], axis = 0)
    test = pd.concat([test,te], axis = 0)

train = train.reset_index(drop=True) 
test = test.reset_index(drop=True)

## Data 지정
x_train, y_train = train.drop('15by1_SI',axis=1), train['15by1_SI']
x_test, y_test = test.drop('15by1_SI',axis=1), test['15by1_SI']

### 5. Model Fitting
: 아래 모형들을 통해 해당 요일의 15일 뒤의 샤프지수를 예측하는 모형을 구축 (Cross-Validation for cv=5)
- Random Forest
- XGBoost
- LightGBM

##### 5-1. Random Forest

In [24]:
### Random Forest Cross Validation for Searching Best Parameter
rf = RandomForestRegressor()

param_rf = {
    "n_estimators": [100,150,200],
    "max_depth": [None,10,30]
}

rf_grid = GridSearchCV(estimator = rf,
                       param_grid = param_rf,
                       cv = 5, 
                       refit = True, 
                       n_jobs = -1)

In [25]:
rf_grid.fit(x_train, y_train)

GridSearchCV(cv=5, estimator=RandomForestRegressor(), n_jobs=-1,
             param_grid={'max_depth': [None, 10, 30],
                         'n_estimators': [100, 150, 200]})

In [28]:
scores = pd.DataFrame(grid_search.cv_results_)
scores[['params','mean_test_score','rank_test_score','split0_test_score','split1_test_score','split2_test_score','split3_test_score','split4_test_score']]

,params,mean_test_score,rank_test_score,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score
0,"{'max_depth': None, 'n_estimators': 100}",-0.083113,8,-0.101341,-0.069171,-0.055680,-0.041318,-0.148056
1,"{'max_depth': None, 'n_estimators': 150}",-0.076284,5,-0.092647,-0.074067,-0.055181,-0.040337,-0.119189
2,"{'max_depth': None, 'n_estimators': 200}",-0.077182,6,-0.100601,-0.060201,-0.048615,-0.038303,-0.138190
3,"{'max_depth': 10, 'n_estimators': 100}",-0.011241,2,-0.042735,-0.000112,0.011866,0.024456,-0.049678
4,"{'max_depth': 10, 'n_estimators': 150}",-0.015550,3,-0.046170,-0.002509,0.013133,0.022717,-0.064923
5,"{'max_depth': 10, 'n_estimators': 200}",-0.009746,1,-0.040033,0.002496,0.015161,0.026650,-0.053004
6,"{'max_depth': 30, 'n_estimators': 100}",-0.087116,9,-0.107161,-0.079172,-0.059683,-0.046262,-0.143300
7,"{'max_depth': 30, 'n_estimators': 150}",-0.078333,7,-0.100720,-0.065904,-0.048920,-0.043567,-0.132555
8,"{'max_depth': 30, 'n_estimators': 200}",-0.075434,4,-0.097894,-0.066944,-0.049773,-0.038386,-0.124172


In [29]:
### Refit Random Forest for Best Hyper-parameter
rf_model = RandomForestRegressor(random_state = 0, max_depth = 10, n_estimators = 200)
rf_model.fit(x_train, y_train)

y_pred = rf_model.predict(x_test)

mse = np.sqrt(mean_squared_error(y_pred, y_test))
a = pd.DataFrame(y_pred,y_test).reset_index()

print('평균제곱근오차 : ', mse)
print('예측값과 상관계수 : ', a.corr().iloc[1,0])

평균제곱근오차 :  0.31612728321999106
예측값과 상관계수 :  0.34334366097749003


##### 5-2. XGBoost

In [14]:
### XGBoost Cross Validation for cv=5
xgb = xgboost.XGBRegressor()

parameters = {
    'max_depth': [None, 10, 30],
    'n_estimators': [100, 150, 200],
    'learning_rate': [0.05, 0.1],
    'colsample_bytree': [0.5, 0.7],
    'subsample': [0.75] 
}

xgb_grid = GridSearchCV(xgb, parameters, cv=5, n_jobs=5, verbose=5)

start = time.time()
xgb_grid.fit(x_train, y_train)
print(time.time() - start)

xgb_scores = pd.DataFrame(xgb_grid.cv_results_)
xgb_scores[['params','mean_test_score', 'rank_test_score', 'split0_test_score','split1_test_score', 'split2_test_score']]

print(xgb_grid.best_score_)
print(xgb_grid.best_params_)

Fitting 5 folds for each of 36 candidates, totalling 180 fits
[CV 4/5] END colsample_bytree=0.5, learning_rate=0.05, max_depth=None, n_estimators=100, subsample=0.75;, score=0.029 total time=   6.2s
[CV 3/5] END colsample_bytree=0.5, learning_rate=0.05, max_depth=None, n_estimators=150, subsample=0.75;, score=0.019 total time=   9.3s
[CV 1/5] END colsample_bytree=0.5, learning_rate=0.05, max_depth=None, n_estimators=200, subsample=0.75;, score=-0.070 total time=  12.6s
[CV 2/5] END colsample_bytree=0.5, learning_rate=0.05, max_depth=10, n_estimators=100, subsample=0.75;, score=-0.021 total time=  11.3s
[CV 3/5] END colsample_bytree=0.5, learning_rate=0.05, max_depth=10, n_estimators=150, subsample=0.75;, score=-0.035 total time=  17.6s
[CV 4/5] END colsample_bytree=0.5, learning_rate=0.05, max_depth=10, n_estimators=200, subsample=0.75;, score=-0.014 total time=  23.7s
[CV 1/5] END colsample_bytree=0.5, learning_rate=0.05, max_depth=30, n_estimators=100, subsample=0.75;, score=-0.089 t

/opt/homebrew/Caskroom/miniforge/base/lib/python3.10/site-packages/joblib/externals/loky/process_executor.py:700: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


[CV 5/5] END colsample_bytree=0.5, learning_rate=0.05, max_depth=None, n_estimators=100, subsample=0.75;, score=-0.008 total time=   6.2s
[CV 4/5] END colsample_bytree=0.5, learning_rate=0.05, max_depth=None, n_estimators=150, subsample=0.75;, score=0.024 total time=   9.4s
[CV 5/5] END colsample_bytree=0.5, learning_rate=0.05, max_depth=None, n_estimators=200, subsample=0.75;, score=-0.044 total time=  12.6s
[CV 4/5] END colsample_bytree=0.5, learning_rate=0.05, max_depth=10, n_estimators=100, subsample=0.75;, score=0.002 total time=  11.3s
[CV 4/5] END colsample_bytree=0.5, learning_rate=0.05, max_depth=10, n_estimators=150, subsample=0.75;, score=-0.008 total time=  17.5s
[CV 1/5] END colsample_bytree=0.5, learning_rate=0.05, max_depth=10, n_estimators=200, subsample=0.75;, score=-0.066 total time=  23.8s
[CV 2/5] END colsample_bytree=0.5, learning_rate=0.05, max_depth=30, n_estimators=100, subsample=0.75;, score=-0.096 total time=  35.6s
[CV 3/5] END colsample_bytree=0.5, learning_

In [417]:
### Refit XGBoost for Best Hyper-Parameter
xgb_best = xgboost.XGBRegressor(colsample_bytree = 0.5,
                                learning_rate = 0.05, 
                                max_depth = None, 
                                n_estimators = 100, 
                                subsample = 0.75)

xgb_best.fit(x_train, y_train)

y_pred_xgb = xgb_best.predict(x_test)

mse_xgb = np.sqrt(mean_squared_error(y_pred_xgb, y_test))
a_xgb = pd.DataFrame(y_pred_xgb,y_test).reset_index()

In [46]:
print('평균제곱근오차 : ', mse_xgb)
print('예측값과 상관계수 : ', a_xgb.corr().iloc[0,1])

평균제곱근오차 :  0.32269003929440526
예측값과 상관계수 :  0.3005944780665503


##### 5-3. LightGBM

In [41]:
### LightGBM Cross Validation for cv=5
lgbm = LGBMRegressor()

parameters = {
    'max_depth': [None, 10, 30],
    'n_estimators': [100, 150, 200],
    'learning_rate': [0.05, 0.1],
    'colsample_bytree': [0.5, 0.7],
    'subsample': [0.75]
}

lgbm_grid = GridSearchCV(lgbm, parameters, cv=5, n_jobs=5, verbose=5)

start = time.time()
lgbm_grid.fit(x_train, y_train)
print(time.time() - start)

_lgbm = pd.DataFrame(lgbm_grid.cv_results_)
scores_lgbm[['params','mean_test_score', 'rank_test_score', 'split0_test_score','split1_test_score', 'split2_test_score']]

print(lgbm_grid.best_score_)
print(lgbm_grid.best_params_)

Fitting 5 folds for each of 36 candidates, totalling 180 fits
27.97442936897278
0.019788780678146912
{'colsample_bytree': 0.5, 'learning_rate': 0.05, 'max_depth': None, 'n_estimators': 100, 'subsample': 0.75}


In [418]:
### Refit LightGBM for Best Hyper-Parameter
lgbm_best = LGBMRegressor(colsample_bytree = 0.5, 
                          learning_rate = 0.05,
                          max_depth = None,
                          n_estimators = 100,
                          subsample = 0.75)

lgbm_best.fit(x_train, y_train)

y_pred_lgbm = lgbm_best.predict(x_test)

mse_lgbm = np.sqrt(mean_squared_error(y_pred_lgbm, y_test))
a_lgbm = pd.DataFrame(y_pred_lgbm,y_test).reset_index()

In [419]:
print('평균제곱근오차 : ', mse_lgbm)
print('예측값과 상관계수 : ', a_lgbm.corr().iloc[0,1])

평균제곱근오차 :  0.3216166838029524
예측값과 상관계수 :  0.27963947179028115


##### 5-4. Prediction : Final Sharpe Index
: 모형 3개 비교 결과 평균제곱근오차가 제일 작고 예측값과의 상관계수가 제일 높은 Random Forest 모형으로 결정

In [370]:
### Final Model : Random Forest at Hyper-paramer 

## Model Setting
final = RandomForestRegressor(random_state = 0, max_depth = 10, n_estimators = 200)

## 2023년 7월 28일 데이터 예측
final_pred = final.predict(real_test)

## 2023년 7월 28일 데이터 Merge
real_test['real_test'] = final_pred
result = pd.merge(real_test,code_dict,left_on = '종목코드',right_on = '종목코드_x',how = 'left')
result = result[['종목코드_y','real_test']]
result.columns = ['종목코드','SI']

### 6. Data Export

In [341]:
### Data Export

## Sharpe Index Prediction Value Export
result.to_csv("C:/Users/kyt28/OneDrive/바탕 화면/submit_data/phase4_result1", index = False, encoding="utf-8-sig")

## Exception Stock Export
ex1 = pd.DataFrame(excep_code1)
ex1.columns = ['종목코드']
ex2 = pd.DataFrame(excep_code2)
ex2.columns = ['종목코드']
excep_result = pd.concat([ex1,ex2],axis=0).reset_index(drop=True)

excep_result.to_csv("C:/Users/kyt28/OneDrive/바탕 화면/submit_data/phase4_result2.csv", index = False, encoding="utf-8-sig")

---------------------------------------------------------------------------------------------------------------------------------------------------------------------

## Phase 5. Final Portfolio
: 모든 결과를 총합하여 최종의 포트폴리오를 제작

### 1. Data Import

In [460]:
### Data Import

## Phase1 & Phase2 & Phase3 Result Merged Data
result1 = pd.read_csv("C:/Users/kyt28/OneDrive/바탕 화면/submit_data/phase3_result.csv")

## Phase4 Sharpe Index Prediction Data
result2 = pd.read_csv("C:/Users/kyt28/OneDrive/바탕 화면/submit_data/phase4_result1.csv")

# Data Sort by Sharpe Index
result2 = result2.sort_values(by = 'SI', ascending = False).reset_index(drop=True)

## Phase4 Exception Data : 여기서 result1과의 교집합은 삭제 (이유 = Phase1이후, Phase4에서 다시 데이터를 불러왔기 때문에 Phase3에서 제외했던 종목도 똑같이 제외)
result3 = pd.read_csv("C:/Users/kyt28/OneDrive/바탕 화면/submit_data/phase4_result2.csv")
exc = set(result1['종목코드']).intersection(set(result3['종목코드']))
result3 = result3[~result3['종목코드'].isin(exc)]

In [461]:
### Rank Index

## Short Portfolio Rank
rest_short_num = result1[result1['순위'] > 1800]['순위'].tolist()[0] - 1801
rest_short = result2.tail(int(rest_short_num))
rest_short['순위'] = np.arange(1801,1801+rest_short_num)
rest_short = rest_short[['종목코드','순위']]

## Long & Rest Portfolio Rank

# rest result2 data
rest_short_code = rest_short['종목코드'].tolist()
result2 = result2[~result2['종목코드'].isin(rest_short_code)].drop('SI',axis=1)

# merge result2 & result3
rest_portfolio = pd.concat([result2,result3],axis=0)
result1_max_num = result1[result1['순위'] <= 200]['순위'].tolist()[-1]
rest_portfolio['순위'] = np.arange(result1_max_num+1,len(rest_portfolio)+result1_max_num+1)

### 2. Data Export

In [463]:
### Merge Total Data
total_portfolio = pd.concat([result1,rest_short,rest_portfolio],axis=0).sort_values(by = '순위').reset_index(drop=True)
total_portfolio.to_csv("C:/Users/kyt28/OneDrive/바탕 화면/submit_data/private_submission.csv", index = False, encoding="utf-8-sig")